In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from nltk.metrics.agreement import AnnotationTask
from sklearn.metrics import cohen_kappa_score as kappa
from sklearn.metrics import accuracy_score as accuracy
from scipy.stats import spearmanr as spearman
pilot_file = "Batch_3198536_batch_results.csv"

In [2]:
data = pd.read_csv(pilot_file)
data.columns = [c.replace('.', '_') for c in data.columns]
#data.head()

In [3]:
def extract_dataframe(data):
    '''
    Input: Pandas csv dataframe obtained from MTurk
    
    Output: Pandas dataframe levelled by (User x Sentenced_ID)
    '''
    data["dicts"] = data["Input_var_arrays"].map(lambda x: json.loads(x))
    global_list = []
    
    for row in data.itertuples():
        for idx, local_dict in enumerate(row.dicts):
            temp_dict = local_dict.copy()
            var_instant = "Answer_instantaneous" + str(idx+1)
            var_instant_c =  "Answer_confidence_instantaneous" + str(idx+1)
            var_start = "Answer_startpt" + str(idx+1)
            var_start_c = "Answer_confidence_startpoint" + str(idx+1)
            var_end = "Answer_endpt" + str(idx+1)
            var_end_c = "Answer_confidence_endpoint" + str(idx+1)
            var_duration = "Answer_duration" + str(idx+1)
            var_duration_c = "Answer_confidence_duration" + str(idx+1)

            temp_dict["is_instant"] = getattr(row, var_instant)
            temp_dict["conf_instant"] = getattr(row, var_instant_c)
            temp_dict["has_start"] = getattr(row, var_start)
            temp_dict["conf_start"] = getattr(row, var_start_c)
            temp_dict["has_end"] = getattr(row, var_end)
            temp_dict["conf_end"] = getattr(row, var_end_c)
            temp_dict["duration"] = getattr(row, var_duration)
            temp_dict["conf_duration"] = getattr(row, var_duration_c)
            temp_dict["worker_id"] = row.WorkerId
            temp_dict["hit_id"] = row.HITId

            global_list.append(temp_dict)
            
    return pd.DataFrame(global_list)


In [4]:
pilot_data = extract_dataframe(data)

#Re-arrange the order 
cols = list(pilot_data.columns)
pilot_data = pilot_data[pilot_data.columns[::-1]].drop('sentence', axis=1)

#pilot_data.replace('na',np.NaN, inplace=True)
pilot_data.head
pilot_data['sent_token'] = pilot_data['sentence_id'] + "_" +\
                            pilot_data['pred_token'].map(lambda x: str(x))
pilot_data.to_csv("pilot_data.csv", index=False)

In [5]:
pilot_data.head()

,worker_id,sentence_id,pred_token,pred,is_instant,id,hit_id,has_start,has_end,duration,conf_start,conf_instant,conf_end,conf_duration,sent_token
0,AQ2U4H38MELRY,en-ud-train.conllu 5784,3,occurred,True,1,3NCN4N1H1GM453MD5NG1QQJZWHXBN7,na,na,na,na,2,na,na,en-ud-train.conllu 5784_3
1,AQ2U4H38MELRY,en-ud-train.conllu 5784,5,processing,False,2,3NCN4N1H1GM453MD5NG1QQJZWHXBN7,true,false,7,3,2,2,2,en-ud-train.conllu 5784_5
2,AQ2U4H38MELRY,en-ud-train.conllu 1010,3,held,False,3,3NCN4N1H1GM453MD5NG1QQJZWHXBN7,false,false,5,1,3,2,0,en-ud-train.conllu 1010_3
3,AQ2U4H38MELRY,en-ud-train.conllu 1010,15,curating,False,4,3NCN4N1H1GM453MD5NG1QQJZWHXBN7,false,false,5,2,4,2,1,en-ud-train.conllu 1010_15
4,AQ2U4H38MELRY,en-ud-train.conllu 5106,20,acted,False,5,3NCN4N1H1GM453MD5NG1QQJZWHXBN7,false,false,5,3,2,3,0,en-ud-train.conllu 5106_20
